# Multi-Dimensional Analysis

This notebook provides a walkthrough of some of the multi-dimensional analysis
capabilities of the `sharrow` library.

In [ ]:
import numpy as np
import xarray as xr

import sharrow as sh

sh.__version__

In [ ]:
# TEST check versions
import packaging

assert packaging.version.parse(xr.__version__) >= packaging.version.parse("0.20.2")

## Example Data

We'll begin by again importing some example data to work with.  We'll be using 
some test data taken from the MTC example in the ActivitySim project, including 
tables of data for households and persons, as well as a set of 
skims containing transportation level of service information for travel around
a tiny slice of San Francisco.

The households and persons are typical tabular data, and 
each can be read in and stored as a `pandas.DataFrame`.

In [ ]:
households = sh.example_data.get_households()
households.head()

In [ ]:
# test households content
assert len(households) == 5000
assert "income" in households
assert households.index.name == "HHID"

In [ ]:
persons = sh.example_data.get_persons()
persons.head()

In [ ]:
assert len(persons) == 8212
assert "household_id" in persons
assert persons.index.name == "PERID"

The skims, on the other hand, are not just simple tabular data, but rather a 
multi-dimensional representation of the transportation system, indexed by origin.
destination, and time of day. Rather than using a single DataFrame for this data,
we store it as a multi-dimensional `xarray.Dataset`.

In [ ]:
skims = sh.example_data.get_skims()
skims

For this example, we'll also load a land use table, that contains some attributes of the alternatives.

In [ ]:
landuse = sh.example_data.get_land_use()
landuse.head()

## Multi-Dimensional Analysis

Now that we're loaded our inputs, let's take a look at 
preparing some data for a workplace location choice simulation model.
This is a different kind of model, and it will use differently shaped data: the decision 
makers (or "choosers") in this model will be the workers, and the alternatives 
will be the various zones included in the land use table.

The workers are only a subset of the persons data we looked at before.  We can identify workers from
values 1 and 2 (full-time employed and part-time employed) in the `'pemploy'` attribute 
of the `persons` table. 

In [ ]:
workers = persons.query("pemploy in [1,2]").rename_axis(index="WORKERID")
workers

As we filter the persons table to just the workers, we also renamed the index from
"PERSONID" to "WORKERID".  This renaming is important for `sharrow`, as it expects dimensions
that have the same name to match, but the workers don't align directly with the persons 
anymore.

For our workplace location choice model, we will want to link in data from our skims,
which can tell us about travel times and costs.  Since we have not yet determined a 
time of day for each worker's work tours, we'll just use the `'AM'` skims for the outbound
leg of a hypothetical work tour, and the `'PM'` skims for the return leg.  Instead of 
trying to select constant skims using the dynamic lookups that sharrow can compile, 
we can just filter the skims down in a static manner before placing them into the data tree.

In [ ]:
skims_am = skims.sel(time_period="AM")
skims_pm = skims.sel(time_period="PM")

## Creating a DataTree Iteratively

The last step in getting ready for this model is building out the relationships between all
this data we've prepared. We'll again use the `DataTree` class to do that, but this time 
we'll demostrate building the tree in stages.  First, we'll create a
base Dataset to be the root data for the tree. We can start by creating an otherwise empty `Dataset` indexed on the two dimensions we want to end up with for analysis, workers and zones.

In [ ]:
base = sh.dataset.from_named_objects(
    workers.index,
    landuse.index,
)

In [ ]:
base

Since our base dataset has
two dimensions, we can specify a dimension order when writing into
a DataTree (the default is alphabetical order).
This ordering will be applied to outputs from the flows later.

In [ ]:
tree = sh.DataTree(base=base, dim_order=("WORKERID", "TAZ"))

In [ ]:
# TEST tree_dest attributes
assert tree.dim_order == ("WORKERID", "TAZ")
assert tree.shape == (4361, 25)

Then, we can progressively build our `DataTree` by adding additional data. 
Each new branch of the tree we want to add using the `add_dataset` command should have a 
unique name, a dataset being attached, and one or more relationship declarations
that describe how the new data attaches.  For example, we can attach the `persons`
data like this:

In [ ]:
tree.add_dataset("person", persons, "base.WORKERID @ person.PERID")

The relationship definition here starts with a dotted name of some data 
dimension already in the tree, an `@` operator to indicating matching by
label in that dimension.

In [ ]:
tree.add_dataset("landuse", landuse, "base.TAZ @ landuse.TAZ")
tree.add_dataset("hh", households, "person.household_id @ hh.HHID")

Unlike in the mode choice work in the previous example, we've already filtered the 
time period dimensions of the skims to be morning and afternoon peak,
so we simply attach the two different parts, linking relationships only
for the remaining dimensions.

In [ ]:
tree.add_dataset(
    "odskims",
    skims_am,
    relationships=(
        "hh.TAZ @ odskims.otaz",
        "base.TAZ @ odskims.dtaz",
    ),
)

tree.add_dataset(
    "doskims",
    skims_pm,
    relationships=(
        "base.TAZ @ doskims.otaz",
        "hh.TAZ @ doskims.dtaz",
    ),
)

## Dynamically Defined Flows 

Although it is convenient to write expressions into a seperately configured 
"spec" file, especially when working with 
ActivitySim, it's not strictly necessary to employ such a file in csv format; 
a simple Python dictionary can also be used to setup a flow.

In [ ]:
definition = {
    "round_trip_dist": "odskims.DIST + doskims.DIST",
    "round_trip_dist_first_mile": "clip(odskims.DIST, 0, 1) + clip(doskims.DIST, 0, 1)",
    "round_trip_dist_addl_miles": "clip(odskims.DIST-1, 0, None) + clip(doskims.DIST-1, 0, None)",
    "size_term": "log(TOTPOP + 0.5*EMPRES)",
}

flow = tree.setup_flow(definition)

Loading from this flow is done the same as before.

In [ ]:
arr = flow.load()
arr

In [ ]:
# TEST
assert arr.shape == (4361, 25, 4)
expected = np.array(
    [
        [
            [0.61, 0.61, 0.0, 4.610157],
            [0.28, 0.28, 0.0, 5.681878],
            [0.56, 0.56, 0.0, 6.368187],
            [0.53, 0.53, 0.0, 5.741399],
            [1.23, 1.23, 0.0, 7.17549],
        ],
        [
            [1.19, 1.19, 0.0, 4.610157],
            [1.49, 1.49, 0.0, 5.681878],
            [1.88, 1.85, 0.03, 6.368187],
            [1.36, 1.36, 0.0, 5.741399],
            [1.93, 1.93, 0.0, 7.17549],
        ],
        [
            [1.19, 1.19, 0.0, 4.610157],
            [1.49, 1.49, 0.0, 5.681878],
            [1.88, 1.85, 0.03, 6.368187],
            [1.36, 1.36, 0.0, 5.741399],
            [1.93, 1.93, 0.0, 7.17549],
        ],
        [
            [0.24, 0.24, 0.0, 4.610157],
            [0.61, 0.61, 0.0, 5.681878],
            [1.01, 1.01, 0.0, 6.368187],
            [0.75, 0.75, 0.0, 5.741399],
            [1.38, 1.38, 0.0, 7.17549],
        ],
        [
            [0.61, 0.61, 0.0, 4.610157],
            [0.28, 0.28, 0.0, 5.681878],
            [0.56, 0.56, 0.0, 6.368187],
            [0.53, 0.53, 0.0, 5.741399],
            [1.23, 1.23, 0.0, 7.17549],
        ],
    ],
    dtype=np.float32,
)

np.testing.assert_array_almost_equal(arr[:5, :5, :], expected)

For the tour mode example above, the tours dataset had only one dimension (TOURIDX),
and so the output of the load function had two dimensions (TOURIDX and expressions).
In this example, the base dataset in the tree has two dimensions (workers and zones)
and so the result from the basic `load` function has *three* dimensions (workers, zones, and expressions).

In [ ]:
arr.shape

Just as we could neatly format the two-dimensional output above as a `pandas.DataFrame`,
so too can we neatly format this three-dimensional output, as a `xarray.DataArray`.

In [ ]:
arr_pretty = flow.load_dataarray()
arr_pretty

In [ ]:
# TEST
assert isinstance(arr_pretty, xr.DataArray)
assert arr_pretty.dims == ("WORKERID", "TAZ", "expressions")
assert arr_pretty.shape == (4361, 25, 4)
assert all(
    arr_pretty.expressions
    == np.array(
        [
            "round_trip_dist",
            "round_trip_dist_first_mile",
            "round_trip_dist_addl_miles",
            "size_term",
        ],
        dtype="<U26",
    )
)

## Linear-in-Parameters Functions

We can also use the `dot` method here with the two dimensional base.
We'll apply a one-dimensional coefficients array, with length three to 
match the three terms in the spec.

In [ ]:
coefs = np.asarray([1.0, 0.1, 0.01, 0.001])
flow.dot(coefs)

The `dot_dataarray` method does the same underlying computational work, but 
yields a well-formatted DataArray intead of just a plain numpy array.

In [ ]:
flow.dot_dataarray(coefs)

## Multinomial Logit Simulation

And we can build and simulate an MNL model directly using the `logit_draws` method.  
To do so we need to
provide the "random" draws exogenously.  Here, we'll sample 10 zones (with
replacement) from the selection of alternatives.

In [ ]:
draws = np.random.default_rng(123).random(size=[4361, 10])

In [ ]:
choices, choice_probs = flow.logit_draws(
    coefficients=coefs,
    draws=draws,
)

In [ ]:
choices

In [ ]:
choice_probs

In [ ]:
# TEST
expected_ch = np.array(
    [
        [5, 8, 8, 9, 9, 18, 19, 19, 19, 20],
        [7, 7, 7, 9, 10, 17, 18, 19, 20, 23],
        [1, 6, 6, 7, 9, 9, 13, 17, 18, 22],
        [8, 9, 9, 18, 18, 19, 19, 19, 19, 20],
        [2, 4, 6, 8, 9, 10, 17, 17, 18, 18],
        # ...,
        [0, 1, 7, 9, 13, 21, 22, 22, 24, 24],
        [0, 5, 5, 6, 8, 8, 18, 21, 22, 22],
        [5, 6, 7, 13, 15, 22, 22, 22, 23, 23],
        [0, 2, 3, 13, 16, 22, 22, 22, 24, 24],
        [7, 8, 8, 11, 14, 14, 16, 21, 22, 22],
    ],
    dtype=np.int32,
)
np.testing.assert_array_equal(choices[:5], expected_ch[:5])
np.testing.assert_array_equal(choices[-5:], expected_ch[-5:])

expected_pr = np.array(
    [
        [
            0.021757,
            0.082092,
            0.082092,
            0.090812,
            0.090812,
            0.239048,
            0.130507,
            0.130507,
            0.130507,
            0.038469,
        ],
        [
            0.063636,
            0.063636,
            0.063636,
            0.103338,
            0.039564,
            0.035372,
            0.10316,
            0.064873,
            0.021167,
            0.031342,
        ],
        [
            0.017309,
            0.052503,
            0.052503,
            0.063636,
            0.103338,
            0.103338,
            0.008113,
            0.035372,
            0.10316,
            0.054565,
        ],
        [
            0.08459,
            0.094525,
            0.094525,
            0.246322,
            0.246322,
            0.134478,
            0.134478,
            0.134478,
            0.134478,
            0.040041,
        ],
        [
            0.006765,
            0.014148,
            0.027726,
            0.082092,
            0.090812,
            0.035121,
            0.082798,
            0.082798,
            0.239048,
            0.239048,
        ],
        # ...,
        [
            0.046512,
            0.039614,
            0.019715,
            0.028343,
            0.031909,
            0.08728,
            0.207882,
            0.207882,
            0.06648,
            0.06648,
        ],
        [
            0.046512,
            0.039726,
            0.039726,
            0.027111,
            0.038968,
            0.038968,
            0.028924,
            0.08728,
            0.207882,
            0.207882,
        ],
        [
            0.039726,
            0.027111,
            0.019715,
            0.031909,
            0.023773,
            0.207882,
            0.207882,
            0.207882,
            0.069134,
            0.069134,
        ],
        [
            0.046512,
            0.036197,
            0.025022,
            0.031909,
            0.03535,
            0.207882,
            0.207882,
            0.207882,
            0.06648,
            0.06648,
        ],
        [
            0.019715,
            0.038968,
            0.038968,
            0.013389,
            0.048031,
            0.048031,
            0.03535,
            0.08728,
            0.207882,
            0.207882,
        ],
    ]
)
np.testing.assert_array_almost_equal(choice_probs[:5], expected_pr[:5])
np.testing.assert_array_almost_equal(choice_probs[-5:], expected_pr[-5:])

In [ ]:
# TEST
choices_darr, choice_probs_darr = flow.logit_draws(
    coefficients=coefs,
    draws=draws,
    as_dataarray=True,
)
assert choices_darr.dims == ("WORKERID", "DRAW")
assert choices_darr.shape == (4361, 10)
np.testing.assert_array_equal(choices_darr[:5], expected_ch[:5])
np.testing.assert_array_equal(choices_darr[-5:], expected_ch[-5:])
assert choice_probs_darr.dims == ("WORKERID", "DRAW")
assert choice_probs_darr.shape == (4361, 10)
np.testing.assert_array_almost_equal(choice_probs_darr[:5], expected_pr[:5])
np.testing.assert_array_almost_equal(choice_probs_darr[-5:], expected_pr[-5:])

It's more common to make many repeated choices for destination choice type models
(e.g. to sample destinations), so there's also a "pick count" feature, that
can summarize the simulation results efficiently.

In [ ]:
choices_, choice_probs_, pick_count = flow.logit_draws(
    coefficients=coefs,
    draws=draws,
    pick_counted=True,
)

If you compare against the non-pick-counted results above, you'll see 
that we get exactly the same choices out, but when choices are repeated
they are aggregated in the resulting arrays.

In [ ]:
choices_

In [ ]:
pick_count

In [ ]:
# TEST pick count results
for i in range(choices.shape[0]):
    uc, pc = np.unique(choices[i], return_counts=True)
    np.testing.assert_array_equal(uc, choices_[i, : len(uc)])
    np.testing.assert_array_equal(np.full(10 - len(uc), -1), choices_[i, len(uc) :])
    np.testing.assert_array_equal(pc, pick_count[i, : len(uc)])
    np.testing.assert_array_equal(np.zeros(10 - len(uc)), pick_count[i, len(uc) :])

In [ ]:
# TEST
choices__darr, choice_probs__darr, pick_count_darr = flow.logit_draws(
    coefficients=coefs,
    draws=draws,
    pick_counted=True,
    as_dataarray=True,
)
assert choices__darr.dims == ("WORKERID", "DRAW")
assert choices__darr.shape == (4361, 10)
assert choice_probs__darr.dims == ("WORKERID", "DRAW")
assert choice_probs__darr.shape == (4361, 10)
assert pick_count_darr.dims == ("WORKERID", "DRAW")
assert pick_count_darr.shape == (4361, 10)

### Accessing Logsums

If you want to also access the MNL logsum values from the choice model, 
adding `logsums=True` will return those values in the fourth position of 
the returned tuple (even if pick counting is disabled, the logsum array
is in the 4th value):

In [ ]:
choices, choice_probs, _, logsums = flow.logit_draws(
    coefficients=coefs,
    draws=draws,
    logsums=True,
)
logsums

In [ ]:
# TEST logsums
expected = np.array(
    [
        5.618349,
        5.701237,
        5.701237,
        5.739875,
        5.618349,
        # ...,
        5.585756,
        5.585756,
        5.585756,
        5.585756,
        5.585756,
    ]
)
np.testing.assert_array_almost_equal(logsums[:5], expected[:5])
np.testing.assert_array_almost_equal(logsums[-5:], expected[-5:])

In [ ]:
# TEST
choices_darr2, choice_probs_darr2, pick_count_nope, logsums_darr = flow.logit_draws(
    coefficients=coefs,
    draws=draws,
    logsums=True,
    as_dataarray=True,
)
assert choices_darr2.dims == ("WORKERID", "DRAW")
assert choices_darr2.shape == (4361, 10)
assert choice_probs_darr2.dims == ("WORKERID", "DRAW")
assert choice_probs_darr2.shape == (4361, 10)
assert pick_count_nope is None
assert logsums_darr.dims == ("WORKERID",)
assert logsums_darr.shape == (4361,)

## Gotchas

When working with multi-dimension outputs, if you don't specify the dimensions ordering
explicitly (as done above) then the output dimensions will be in lexicographic order 
according to the unicode binary representations of the dimension names.  This is similar
to alphabetical ordering, except all uppercase letters come before lower case letters. 


In [ ]:
tree_unordered = sh.DataTree(
    base=base,
    person=persons,
    landuse=landuse,
    hh=households,
    odskims=skims_am,
    doskims=skims_pm,
    relationships=(
        "base.WORKERID @ person.PERID",
        "base.TAZ @ landuse.TAZ",
        "person.household_id @ hh.HHID",
        "hh.TAZ @ odskims.otaz",
        "base.TAZ @ odskims.dtaz",
        "base.TAZ @ doskims.otaz",
        "hh.TAZ @ doskims.dtaz",
    ),
)

In [ ]:
# TEST tree_unordered attributes
assert tree_unordered.dim_order is None
assert tree_unordered.shape == (25, 4361)

In [ ]:
flow_unordered = tree_unordered.setup_flow(definition)
arr_unordered = flow_unordered.load_dataarray()
arr_unordered.dims

In [ ]:
# TEST flow_unordered
assert arr_unordered.dims == ("TAZ", "WORKERID", "expressions")
assert arr_unordered.shape == (25, 4361, 4)